In [1]:
import mido
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd
import time

In [2]:
def getData(data_folder):
    """
    Walk through a folder, modifing each mid to fit in 128 ticks per beat and be only a piano channel
    :param data_folder: the base data folder used in the model, still nonrepresentative
    :return: it saves a midfile in piano format and other in paino+128tpb
    """
    counter = 0 
    for dirpath, _, filenames in os.walk(data_folder):
        for File in filenames:
            path = os.path.join(dirpath, File)
            mid_copy = mido.MidiFile()
            
            try:
                mid = mido.MidiFile(path)
                tpb = mid.ticks_per_beat
                duration = int(mid.length)

                mid_copy.type = 1
                mid_copy.ticks_per_beat = tpb

                if 10 < duration < 600:
                    track = mido.merge_tracks(mid.tracks)

                    for msg in list(track):
                        if not msg.is_meta and msg.channel != 0 and msg.type == "note_on":
                            msg.velocity = 0
                            
                mid_copy.tracks.append(track)
                mid_copy.save(f"OnlyPiano/{File}")
            
                mid_copy.ticks_per_beat = 128
                mid_copy.save(f"OnlyPiano_TPB128/{File}") 
                
            except:
                print(path)
                continue
            
            counter+=1
            if counter %250 ==0:
                print(counter)

In [3]:
start_time = time.time()
# getData('Dataset')
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---
